In [191]:
import numpy as np
import pandas as pd
import requests
from selenium import webdriver
import re
import math
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool
from time import time

In [192]:
url_template = "https://myshows.me/search/all/?page={}"

In [193]:
def get_pages_count(item):
    items_in_table = 30
    count = item.text
    count = re.sub(re.compile(r'\s+'), '', count)
    return math.ceil(int(count) / items_in_table)
    

In [194]:
def get_serials_on_page(page_id):
    global url_template
    serial_list_url = url_template.format(page_id)
    serial_list_html = requests.get(serial_list_url).text.strip()
    soup = BeautifulSoup(serial_list_html, 'html.parser')
    print(page_id)
    serials = {}
    table = soup.findAll('table', class_='catalogTable')
    items = table[1].findAll('tr')[1:]
    for item in items:
        serial_info = item.find('td').find('a')
        serial_name = serial_info.text.strip()
        serial_url = serial_info.attrs['href']
        serials[serial_name] = re.findall(r'\d+', serial_url)[0]
    return serials

In [195]:
def get_url_list():
    global url_template
    serial_list_url = url_template.format(0)
    serial_list_html = requests.get(serial_list_url).text.strip()
    soup = BeautifulSoup(serial_list_html, 'html.parser')
    pages_count = get_pages_count(soup.find('span', class_='cFirm'))
    
    url_list = []
    t = ThreadPool(8)
    result = t.map(get_serials_on_page, range(pages_count))
    t.close()
    t.join()
    
    return result

In [ ]:
%%time
serial_list = get_url_list()

In [157]:
serials = serial_list[0]
for serial_dict in serial_list[1:]:
    serials.update(serial_dict)

data = pd.DataFrame(list(serials.items()), columns=['Name', 'ID'])
with open('serials.csv', mode='w', encoding='utf-8') as f_csv:
    data.to_csv(f_csv, index=False)